# Model training

In [1]:
import pandas as pd
from models import *
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)


In [2]:
X = pd.read_csv('data/processed/X_processed2.csv')
y = pd.read_csv('data/processed/y.csv')


In [3]:
X.columns

Index(['read', 'write', 'attended_school', 'highest_diploma', 'preschool',
       'lives_with_partner', 'lives_with_mother', 'mother_education',
       'mother_alive', 'lives_with_father', 'father_education', 'father_alive',
       'age', 'male', 'not_enrolled_for_economic_reasons', 'marital_status_1',
       'marital_status_2', 'marital_status_3', 'marital_status_4',
       'marital_status_5', 'family_role_1', 'family_role_2', 'family_role_3',
       'family_role_4', 'family_role_6', 'family_role_7', 'family_role_8',
       'family_role_9', 'family_role_10'],
      dtype='object')

# Models

In [ ]:
rf = random_forest(X, y['poverty_score'])4

Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Best cross-validation accuracy: 0.20


Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}
Best cross-validation accuracy: 0.20

In [5]:
hgb = hist_grad_boost(X, y['poverty_score'])

Best parameters: {'l2_regularization': 2.0, 'learning_rate': 0.01, 'max_depth': 3, 'max_iter': 100, 'min_samples_leaf': 50}
Best cross-validation accuracy: 0.20


In [7]:
nn = neural_network(X, y['poverty_score'])

Best parameters: {'activation': 'tanh', 'hidden_layer_sizes': (200,), 'max_iter': 1000, 'solver': 'sgd'}
Best cross-validation accuracy: 0.21


In [8]:
knn = knn_classifier(X, y['poverty_score'])

Best parameters: {'algorithm': 'ball_tree', 'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'uniform'}
Best cross-validation accuracy: 0.16


In [ ]:
estimators = [
    ('hgb', hgb),
    ('nn', nn),
    ('rf', rf)
]

stacked_model = stacking(X, y['poverty_score'], estimators)

Stacked model cross-validation accuracy: 0.20 (+/- 0.03)
